In [12]:
import torch
from transformers import MBartForConditionalGeneration, MBart50Tokenizer

model_path = "models/"

device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
print(torch.cuda.is_available())

sl = "en_XX"
tl = "ta_IN"

model = MBartForConditionalGeneration.from_pretrained(model_path + "mbart-large-50-many-to-many-mmt").to(device)
tokenizer_EnTa = MBart50Tokenizer.from_pretrained(model_path + "mbart-large-50-many-to-many-mmt", src_lang=sl, tgt_lang=tl)
tokenizer_TaEn = MBart50Tokenizer.from_pretrained(model_path + "mbart-large-50-many-to-many-mmt", src_lang=tl, tgt_lang=sl)


True


In [20]:
import sys

idx = 0

def preprocess(text, tokenizer, tl):
  global idx
  idx += 1
  sys.stdout.write("\r" + f"{idx}/{len(df)}")
  inputs = tokenizer(text, truncation=True, max_length=200, padding="max_length", return_tensors="pt")
  inputs = inputs.to(device)
  generated_tok = model.generate(**inputs, forced_bos_token_id=tokenizer.lang_code_to_id[tl])

  return tokenizer.batch_decode(generated_tok, skip_special_tokens=True)[0]

In [21]:
import pandas as pd

data_path = "cleaned_snopes.csv"
df = pd.read_csv(data_path)
# df["claim"] = df["claim"].apply(preprocess, args=(tokenizer_EnTa, tl))
# df.to_csv("snopes_tamil.csv", encoding="utf-8", index=None)

df = pd.read_csv("snopes_tamil.csv")
df["claim"] = df["claim"].apply(preprocess, args=(tokenizer_TaEn, sl))
df.to_csv("snopes_backtranslation_TaEn.csv", encoding="utf-8", index=None)

1/4433

/home2/s4231317/virtual_env/HC/lib/python3.10/site-packages/transformers/generation/utils.py:1346: UserWarning: Using `max_length`'s default (200) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


4433/4433

In [ ]:
from sklearn.model_selection import train_test_split
from transformers import DataCollatorForSeq2Seq
from tqdm import tqdm
import torch

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

x_ids = torch.as_tensor([ten["input_ids"] for ten in tqdm(tok_data)])
y_ids = torch.as_tensor([ten["labels"] for ten in tqdm(tok_data)])

print(x_ids.shape, y_ids.shape)

100%|██████████| 50001/50001 [00:00<00:00, 1671976.80it/s]


torch.Size([50001, 128]) torch.Size([50001, 128])


In [ ]:
def convert_to_dict(data, lang):
  d = {}
  for idx, dat in enumerate(data[:6]):
    d[idx] = {lang: data[idx]}

  return d

x_train, x_test, y_train, y_test = train_test_split(x_ids, y_ids, test_size=0.25)
x_test, x_val, y_test, y_val = train_test_split(x_test, y_test, test_size=0.5) # validation

print(x_train.shape, x_test.shape, x_val.shape)

x_train = convert_to_dict(x_train, s_lang)
x_test = convert_to_dict(x_test, s_lang)
x_val = convert_to_dict(x_val, s_lang)

y_train = convert_to_dict(y_train, t_lang)
y_test = convert_to_dict(y_test, t_lang)
y_val = convert_to_dict(y_val, t_lang)

# train = [{"id": str(idx), "translation": {s_lang: x_train, t_lang: y_train}} for idx in ids][0]
# test = [{"id": str(idx), "translation": {s_lang: x_test, t_lang: y_test}} for idx in ids][0]

torch.Size([37500, 128]) torch.Size([6250, 128]) torch.Size([6251, 128])


In [ ]:
class Data(torch.utils.data.Dataset):
  def __init__(self, x, y):
    self.x = x
    self.y = y

  def __getitem__(self, idx):
    item = {key: torch.as_tensor(val[idx]) for key, val in self.x.items()}
    print(item)
    item["labels"] = torch.as_tensor(self.y[idx])
    return item

  def __len__(self):
    return len(self.y)

train = Data(x_train, y_train)
test = Data(x_test, y_test)
val = Data(x_val, y_val)

In [ ]:
!pip install -q evaluate sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 15.3 MB/s eta 0:00:00


In [ ]:
import numpy as np
import evaluate

metric = evaluate.load("sacrebleu")

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds

    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}

    return result

In [ ]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

training_args = Seq2SeqTrainingArguments(
    output_dir=f"{s_lang}-{t_lang}_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=2,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=False,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train,
    eval_dataset=test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:1                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:1664 in train                    │
│                                                                                                  │
│   1661 │   │   inner_training_loop = find_executable_batch_size(                                 │
│   1662 │   │   │   self._inner_training_loop, self._train_batch_size, args.auto_find_batch_size  │
│   1663 │   │   )                                                                                 │
│ ❱ 1664 │   │   return inner_training_loop(                                                       │
│   1665 │   │   │   args=args,                                                                    │
│   1666 │   │   │   resume_from_checkpoint=resume_from_checkpoint,                                │
│   1667 │   │   │   trial=trial,                                                                  │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:1909 in _inner_training_loop     │
│                                                                                                  │
│   1906 │   │   │   │   rng_to_sync = True                                                        │
│   1907 │   │   │                                                                                 │
│   1908 │   │   │   step = -1                                                                     │
│ ❱ 1909 │   │   │   for step, inputs in enumerate(epoch_iterator):                                │
│   1910 │   │   │   │   total_batched_samples += 1                                                │
│   1911 │   │   │   │   if rng_to_sync:                                                           │
│   1912 │   │   │   │   │   self._load_rng_state(resume_from_checkpoint)                          │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:633 in __next__           │
│                                                                                                  │
│    630 │   │   │   if self._sampler_iter is None:                                                │
│    631 │   │   │   │   # TODO(https://github.com/pytorch/pytorch/issues/76750)                   │
│    632 │   │   │   │   self._reset()  # type: ignore[call-arg]                                   │
│ ❱  633 │   │   │   data = self._next_data()                                                      │
│    634 │   │   │   self._num_yielded += 1                                                        │
│    635 │   │   │   if self._dataset_kind == _DatasetKind.Iterable and \                          │
│    636 │   │   │   │   │   self._IterableDataset_len_called is not None and \                    │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:677 in _next_data         │
│                                                                                                  │
│    674 │                                                                                         │
│    675 │   def _next_data(self):                                                                 │
│    676 │   │   index = self._next_index()  # may raise StopIteration                             │
│ ❱  677 │   │   data = self._dataset_fetcher.fetch(index)  # may raise StopIteration              │
│    678 │   │   if self._pin_memory:                        

In [ ]:
# sl = "English"
# tl = "French"
# prefix = f"translate {sl} to {tl}: "

# text = prefix + "My name is Eduard"

# inputs = tokenizer(text, return_tensors="pt").input_ids
# outputs = model.generate(inputs, max_new_tokens=40, do_sample=True, top_k=30, top_p=0.95)
# print(tokenizer.decode(outputs[0], skip_special_tokens=True))